## 성능 개선판 word2vec 학습모델 구현

### CBOW 모델 구현

In [1]:
from nn_layers import Embedding, NegativeSamplingLoss, Adam, Trainer
from mynlp import preprocess, most_similar, create_contexts_target
import numpy as np

In [2]:
class CBOW :
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V,H = vocab_size, hidden_size
        
        # 가중치 초기화
        W_in = 0.01*np.random.randn(V,H).astype('f') 
        W_out = 0.01*np.random.randn(V,H).astype('f')  # (H,V) 가 아님에 주의
        
        # 계층 생성
        self.in_layers = []
        for i in range(2*window_size):   # window_size = 5일 경우 , i : 0~9 (10회 반복)    
            layer = Embedding(W_in)      # Embedding 계층 사용
            self.in_layers.append(layer)
        
        self.ns_loss_layer = NegativeSamplingLoss(W_out,corpus,power=0.75,sample_size=5)

        # 모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss_layer]
        self.params ,self.grads = [],[]
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
            
        # 인스턴스 변수 단어의 분산 표현을 저장한다.    
        self.word_vecs = W_in  
        
    def forward(self,contexts,target):  # contexts : (100,10)
        h = 0
        for i,layer in enumerate(self.in_layers): # 10회
            h += layer.forward(contexts[:,i])     # contexts의 1개 컬럼만 추출하여 Embedding층에 idx로 전달
            
        h *= 1/len(self.in_layers)  # h*(1/10) : 10으로 나눔
        loss = self.ns_loss_layer.forward(h,target)
        return loss
    
    def backward(self, dout=1):
        dout = self.ns_loss_layer.backward(dout)
        dout *= 1/len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

### CBOW 모델 학습 코드

In [3]:
from dataset import ptb
import pickle

# 하이퍼 파라미터 설정
window_size = 5
# window_size = 2

hidden_size = 100
batch_size = 100
max_epoch = 10  # 10회 이상

# 데이터 읽기
# 전체 데이터 모두 사용시  # 전체 데이터로 epoch 10회 학습 ==> '약 10시간 소요'
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
print(vocab_size)  # 10000
print(len(corpus)) # 929589

10000
929589


In [4]:
# PTB 데이터 중 일부만 사용시 :  50000 corpus
corpus, word_to_id, id_to_word = ptb.load_data('train')

corpus_size = 50000
corpus = corpus[:corpus_size]

vocab_size = int(max(corpus) + 1)  # 5276

temp1,temp2 = {},{}
for k in range(vocab_size):
    word1= list(word_to_id.keys())[k]    
    id1 = list(word_to_id.values())[k] 
    temp1[word1] = id1
    
    word2= list(id_to_word.keys())[k]    
    id2 = list(id_to_word.values())[k] 
    temp2[word2] = id2
    
word_to_id = temp1
id_to_word =temp2

print(len(corpus))
print(vocab_size)      # 5276 
print(len(word_to_id))
print(len(id_to_word))

50000
5276
5276
5276


In [5]:
contexts,target = create_contexts_target(corpus,window_size)
# print(corpus[:100])
# print(contexts[:100])
# print(target[:100])

In [ ]:
# 모델 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model,optimizer)

# 학습 : 50000개 일때 약 7분 소요
trainer.fit(contexts, target, max_epoch, batch_size)

| 에폭 1 |  반복 1 / 499 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 21 / 499 | 시간 1[s] | 손실 4.16
| 에폭 1 |  반복 41 / 499 | 시간 2[s] | 손실 4.15
| 에폭 1 |  반복 61 / 499 | 시간 4[s] | 손실 4.11
| 에폭 1 |  반복 81 / 499 | 시간 6[s] | 손실 4.02
| 에폭 1 |  반복 101 / 499 | 시간 7[s] | 손실 3.86
| 에폭 1 |  반복 121 / 499 | 시간 9[s] | 손실 3.67
| 에폭 1 |  반복 141 / 499 | 시간 10[s] | 손실 3.48
| 에폭 1 |  반복 161 / 499 | 시간 12[s] | 손실 3.34
| 에폭 1 |  반복 181 / 499 | 시간 13[s] | 손실 3.19
| 에폭 1 |  반복 201 / 499 | 시간 15[s] | 손실 3.09
| 에폭 1 |  반복 221 / 499 | 시간 16[s] | 손실 3.00
| 에폭 1 |  반복 241 / 499 | 시간 17[s] | 손실 2.93
| 에폭 1 |  반복 261 / 499 | 시간 19[s] | 손실 2.88
| 에폭 1 |  반복 281 / 499 | 시간 20[s] | 손실 2.85
| 에폭 1 |  반복 301 / 499 | 시간 22[s] | 손실 2.80
| 에폭 1 |  반복 321 / 499 | 시간 24[s] | 손실 2.75
| 에폭 1 |  반복 341 / 499 | 시간 25[s] | 손실 2.75
| 에폭 1 |  반복 361 / 499 | 시간 28[s] | 손실 2.73
| 에폭 1 |  반복 381 / 499 | 시간 30[s] | 손실 2.72
| 에폭 1 |  반복 401 / 499 | 시간 32[s] | 손실 2.71
| 에폭 1 |  반복 421 / 499 | 시간 34[s] | 손실 2.69
| 에폭 1 |  반복 441 / 499 | 시간 37[s] | 손실 2.69
|

In [ ]:
trainer.plot()

In [ ]:
# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs

params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'  # 저장할 파일
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1) # 저장

In [ ]:
# W_in :단어의 분산 표현
print(model.word_vecs.shape)
model.word_vecs